In [1]:
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import time
from scipy import optimize
%matplotlib inline

In [2]:
np.random.seed = 420

# Gradient descent method
Since matrix $P$ is given in the problem statement, we will set it uniformly distributed. Let $n = 4$ and $m = 5$.

In [3]:
n = 4
m = 5
P = np.array([np.random.uniform(size=n) for x in np.zeros(m)])
P /= P.sum(axis=0)
c_t = np.array([-np.sum(x * np.log2(x)) for x in P.T])
print(P)

[[ 0.21835855  0.33189874  0.36619697  0.26443389]
 [ 0.17575225  0.03124171  0.13026456  0.11555428]
 [ 0.2897017   0.05940226  0.25613704  0.14353018]
 [ 0.16308107  0.22573601  0.12051723  0.27849854]
 [ 0.15310643  0.35172127  0.1268842   0.19798311]]


In [4]:
def f(x):
    #get projection of x
    x_ = euclidean_proj_simplex(x)
    y = P @ x_
    if (np.min(y) <= 0):
        return np.inf
    return c_t @ x_ + np.sum(y * np.log(y) / np.log(2))

def grad_f(x):
    #get projection of x
    x_ = euclidean_proj_simplex(x)
    y = P @ x_
    if (np.min(y) <= 0):
        raise ValueError
    grad = c_t.copy()
    tmp = []
    for i in range(m):
        tmp.append(P[i] * (np.log(P[i] @ x_) + 1) / np.log(2))
    tmp_sum = np.sum(np.array(tmp), axis=0)
    return grad + tmp_sum

In [5]:
def euclidean_proj_simplex(v, s=1):
    n, = v.shape  
    if v.sum() == s and np.alltrue(v >= 0):
        return v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, n+1) > (cssv - s))[0][-1]
    theta = (cssv[rho] - s) / (rho + 1.0)
    w = (v - theta).clip(min=0)
    return w

In [6]:
class StoppingCriteria:
    def __init__(self, max_iterations=np.inf, min_grad_norm=0):
        self.max_iterations = max_iterations
        self.min_grad_norm = min_grad_norm
    
    def __call__(self, state):
        cur_iterations = state['iterations']
        cur_grad_norm = np.linalg.norm(state['cur_grad'], ord=2)
        dif_x = np.linalg.norm(state['x'] - state['prev_x'], ord=2)
        return (cur_iterations >= self.max_iterations or cur_grad_norm <= self.min_grad_norm or dif_x <= self.min_grad_norm)

In [7]:
class StepSearchFastestTernary:
    def __init__(self, precision):
        self.precision = precision
        self.left = 0
        self.right = None
        
    def __update_starting_points(self, state, init_kpower=-2):
        k_power = init_kpower
        f = state['f']
        x = state['x']
        grad_f = state['cur_grad']
        dx = - grad_f
        while f(x + 2**k_power * dx) > f(x + 2**(k_power + 1) * dx):
            k_power += 1
        if k_power == init_kpower:
            self.left = 0
        else:
            self.left = 2**(k_power - 1)
        self.right = 2**(k_power + 1)
            
            
    def __call__(self, state):
        f = state['f']
        x = state['x']
        cur_grad = state['cur_grad']
        dx = -cur_grad
        
        self.__update_starting_points(state) # update self.left and self.right
        
        right = self.right
        left = self.left
        
        while True:
            if abs(right - left) < self.precision:
                return (left + right)/2

            left_div = left + (right - left)/3
            right_div = right - (right - left)/3

            f_left = f(x + left_div * dx)
            f_right = f(x + right_div * dx)
            
            if f_left == np.inf:
                right = right_div
            else:
                if f_left < f_right:
                    right = right_div
                else:
                    left = left_div
        

In [8]:
class GradientDescentMethod:
    def __init__(self, t_search, stopping_criteria):
        self.t_search = t_search
        self.stopping_criteria = stopping_criteria
    
    def fit(self, f, grad_f, x_0):
        x = x_0.copy()
        state = dict()
        state['f'] = f
        state['grad_f'] = grad_f
        state['x'] = x
        # hardcoded xD
        state['prev_x'] = np.ones(n) / n
        
        state['iterations'] = 0
        state['time'] = time.time()
        while True:
            state['cur_grad'] = grad_f(state['x'])
            if self.stopping_criteria(state):
                break
            t = self.t_search(state)
            state['prev_x'] = state['x'].copy()
            state['x'] -= t * state['cur_grad']
            #take projection on simplex
            state['x'] = euclidean_proj_simplex(state['x'])
            state['iterations'] += 1
            
        state['time'] = time.time() - state['time']
        return state

In [9]:
stopping_criteria = StoppingCriteria(min_grad_norm=1e-7)
t_search = StepSearchFastestTernary(precision=1e-7)
grad = GradientDescentMethod(t_search=t_search, stopping_criteria=stopping_criteria)
x_0 = np.random.uniform(low=0, high=1, size=n)
x_0 /= np.sum(x_0)
state = grad.fit(f, grad_f, x_0)
print('x_0 = ', x_0)
print('f_min =', f(state['x']))
print('time = ', state['time'])
print('x = ',state['x'])

x_0 =  [ 0.13952584  0.39651145  0.33776761  0.12619511]
f_min = -0.141671195045
time =  0.22211384773254395
x =  [ 0.46340459  0.53659541  0.          0.        ]


In [10]:
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1},
        {'type': 'ineq', 'fun': lambda x: x[0]},
        {'type': 'ineq', 'fun': lambda x: x[1]},
       {'type': 'ineq', 'fun': lambda x: x[2]},
       {'type': 'ineq', 'fun': lambda x: x[3]})

In [11]:
print(optimize.minimize(f, x_0, method='SLSQP',
               constraints=cons))

     fun: -0.1416711939597448
     jac: array([ -2.42292881e-05,   2.42590904e-05,   1.13973022e-03,
         6.89630210e-02])
 message: 'Optimization terminated successfully.'
    nfev: 48
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([  4.63359779e-01,   5.36640221e-01,  -1.86482774e-17,
         8.67361738e-19])
